In [1]:
# tham khảo: https://viblo.asia/p/phan-loai-van-ban-tu-dong-bang-machine-learning-nhu-the-nao-4P856Pa1ZY3
# https://pypi.org/project/pyvi/
!pip install pyvi

     |████████████████████████████████| 8.5 MB 7.6 MB/s 
     |████████████████████████████████| 965 kB 41.3 MB/s 


In [2]:
!git clone https://github.com/duyvuleo/VNTC

Cloning into 'VNTC'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), done.
Checking out files: 100% (15/15), done.


In [3]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

In [4]:
import os
from google.colab import output 
def get_data(folder_path):
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-16") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)
                lines = ' '.join(lines)
                lines = ViTokenizer.tokenize(lines)

                X.append(lines)
                y.append(path)

                output.clear()

    return X, y


In [ ]:
!unrar x "/content/VNTC/Data/10Topics/Ver1.1/Train_Full.rar"

In [9]:
train_path = "/content/Train_Full"

In [10]:
X_data, y_data = get_data(train_path)

100%|██████████| 10/10 [06:46<00:00, 40.60s/it]


In [12]:
import pickle
pickle.dump(X_data, open(os.path.join(train_path, "X_data.pkl"), 'wb'))
pickle.dump(y_data, open(os.path.join(train_path, "y_data.pkl"), 'wb'))

In [ ]:
!unrar x "/content/VNTC/Data/10Topics/Ver1.1/Test_Full.rar"

In [21]:
test_path = "/content/Test_Full"

In [22]:
X_test, y_test = get_data(test_path)

100%|██████████| 10/10 [10:31<00:00, 63.18s/it]


In [24]:
print(type(X_test))

<class 'list'>


In [29]:
pickle.dump(X_data, open(os.path.join(train_path, "X_test.pkl"), 'wb'))
pickle.dump(y_data, open(os.path.join(train_path, "y_test.pkl"), 'wb'))

In [30]:
import pickle
import os

X_data = pickle.load(open(os.path.join(train_path, "X_data.pkl"), 'rb'))
y_data = pickle.load(open(os.path.join(train_path, "y_data.pkl"), 'rb'))

X_test = pickle.load(open(os.path.join(train_path, "X_test.pkl"), 'rb'))
y_test = pickle.load(open(os.path.join(train_path, "y_test.pkl"), 'rb'))

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names_out())

print(X.toarray())

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_data)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(X_data)
X_test_count = count_vect.transform(X_test)

Word level

In [ ]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(X_data) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(X_data)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(X_test)

Ngram level

In [ ]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(X_data)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(X_test)

In [ ]:
# ngram-char level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(X_data)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_data)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(X_test)

Giảm chiều ma trận

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)

X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

In [ ]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)

In [ ]:
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_data_tfidf_ngram_char)

X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)


In [ ]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
y_data_n = encoder.fit_transform(y_data)
y_test_n = encoder.fit_transform(y_test)

In [ ]:
encoder.classes_

In [ ]:
X_data_tfidf

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=3):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))


In [ ]:
from sklearn import naive_bayes
train_model(naive_bayes.MultinomialNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

# kết quả
# Train accuracy:  0.880031596616529
# Validation accuracy:  0.8690758293838863
# Test accuracy:  0.8650666031405714

In [ ]:
# train_model(naive_bayes.MultinomialNB(), X_data_tfidf_ngram_svd, y_data, X_test_tfidf_ngram_svd, y_test, is_neuralnet=False)

# train_model(naive_bayes.MultinomialNB(), X_data_tfidf_ngram_char_svd, y_data, X_test_tfidf_ngram_char_svd, y_test, is_neuralnet=False)

In [ ]:
from sklearn import naive_bayes

train_model(naive_bayes.BernoulliNB(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

# kết quả thu được:
# Train accuracy:  0.8485995457986374
# Validation accuracy:  0.8293838862559242
# Test accuracy:  0.8531554602664125


In [ ]:
from sklearn import naive_bayes

train_model(naive_bayes.BernoulliNB(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

# kết quả thu được:
# Train accuracy:  0.8087746437152354
# Validation accuracy:  0.8033175355450237
# Test accuracy:  0.8143449864014453


In [ ]:
from sklearn import linear_model

train_model(linear_model.LogisticRegression(), X_data_tfidf, y_data, X_test_tfidf, y_test, is_neuralnet=False)

# kết quả thu được:
# Train accuracy:  0.9473060593094823
# Validation accuracy:  0.9167654028436019
# Test accuracy:  0.9207511960772636


In [ ]:
from sklearn import linear_model

train_model(linear_model.LogisticRegression(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

# kết quả thu được:
# Train accuracy:  0.9023467070401211
# Validation accuracy:  0.8927725118483413
# Test accuracy:  0.9046314493875688

In [ ]:
from sklearn import svm

train_model(svm.SVC(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

# kết quả thu được:
# Train accuracy:  0.43359773557581544
# Validation accuracy:  0.4277251184834123
# Test accuracy:  0.3908840053203105

In [ ]:
from sklearn import ensemble

train_model(ensemble.RandomForestClassifier(), X_data_tfidf_svd, y_data, X_test_tfidf_svd, y_test, is_neuralnet=False)

# kết quả thu được:
# Train accuracy:  0.9962479017871836
# Validation accuracy:  0.8311611374407583
# Test accuracy:  0.834435114049193
